In [2]:
## import modules
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import dask.dataframe as dd
import dask.array as da
import dask.delayed
#from dask.dot import dot_graph

In [3]:
print("My working directory:\n" + os.getcwd())
# Set Working Directory (if needed)
os.chdir(r"/Users/wenhuizeng/Library/Mobile Documents/com~apple~CloudDocs/high performance/week7/healthcare2")
print("My new working directory:\n" + os.getcwd())

My working directory:
/Users/wenhuizeng/High Performance
My new working directory:
/Users/wenhuizeng/Library/Mobile Documents/com~apple~CloudDocs/high performance/week7/healthcare2


## How common is depression in our patient population?
i. This could be implemented a few different ways. You could simply
consider everyone with at least one visit outpatient visit ‘at-risk’ in the denominator, then calculate if they have ever been diagnosed with depression. This is the simplest approach and fine to use for this assignment.

In [4]:
atrisk_dd = dd.read_csv(r'OutpatientVisit.csv')
num_at_risk = atrisk_dd.PatientID.unique().count().compute()

In [5]:
atrisk_dd.head()

,VisitID,StaffID,PatientID,VisitDate,ICD10_1,ICD10_2,ICD10_3,ClinicCode
0,1,46,1,2013-08-10,E10621,K269,NaN,15
1,2,50,1,2013-12-02,K269,E10621,NaN,55
2,3,13,1,2014-06-29,E10621,K269,NaN,1
3,4,23,1,2014-09-19,K269,E10621,NaN,3
4,5,9,1,2015-05-29,K269,E10621,NaN,5


In [6]:
disease_dd = dd.read_csv(r'DiseaseMap.csv')
depression_code = disease_dd.loc[disease_dd.Condition=="Depression",['ICD10']]
depression_list = depression_code.ICD10.unique().compute()
##dd.melt long.ICD10.str.contain('all the code')

In [7]:
disease_dd.head()

,DiseaseMapID,ICD10,Condition
0,3375,F1010,Alcohol
1,3376,F10120,Alcohol
2,3377,F10121,Alcohol
3,3378,F10129,Alcohol
4,3379,F1014,Alcohol


In [8]:
depression_list = depression_list.values.tolist()
depression_list 

['F320',
 'F321',
 'F322',
 'F323',
 'F328',
 'F329',
 'F330',
 'F331',
 'F332',
 'F333',
 'F338',
 'F339',
 'F341',
 'F4321']

In [13]:
atrisk_dd['depression'] = atrisk_dd.ICD10_1.isin(depression_list)|atrisk_dd.ICD10_2.isin(depression_list)|atrisk_dd.ICD10_3.isin(depression_list)


In [14]:
depression_num = atrisk_dd.loc[atrisk_dd.depression==True,'PatientID'].unique().count().compute()

In [15]:
depression_rate = depression_num/num_at_risk
depression_rate

0.10718648208469056

# Are depressed patients more likely to die than non-depressed patients?
i. Again, this could be implemented a few different ways. You could consider the outcome to be mortality at any point and ignore
differential follow-up time for this assignment.

In [16]:
mortality_dd = dd.read_csv(r'Mortality.csv')

In [17]:
atrisk_dd.head()

,VisitID,StaffID,PatientID,VisitDate,ICD10_1,ICD10_2,ICD10_3,ClinicCode,depression
0,1,46,1,2013-08-10,E10621,K269,NaN,15,False
1,2,50,1,2013-12-02,K269,E10621,NaN,55,False
2,3,13,1,2014-06-29,E10621,K269,NaN,1,False
3,4,23,1,2014-09-19,K269,E10621,NaN,3,False
4,5,9,1,2015-05-29,K269,E10621,NaN,5,False


In [18]:
depression = atrisk_dd.drop_duplicates(subset=['PatientID']).compute()

In [19]:
depression.head()

,VisitID,StaffID,PatientID,VisitDate,ICD10_1,ICD10_2,ICD10_3,ClinicCode,depression
0,1,46,1,2013-08-10,E10621,K269,NaN,15,False
14,15,14,2,2006-07-04,C4650,O10019,NaN,40,False
20,21,48,3,2011-12-13,B20,O1092,NaN,53,False
37,38,27,4,2010-03-07,J452,E131,NaN,53,False
53,54,50,5,2007-02-07,O10013,NaN,NaN,55,False


In [20]:
mortality_dd.head()

,PatientID,DateOfDeath
0,2,2008-11-02
1,4,2016-01-17
2,13,2010-11-27
3,17,2008-11-27
4,18,2010-09-22


In [21]:
#merge_data = depression(mortality_dd, on='PatientID')
#mortality_dd.join(depression, on='PatientID')    
#merge_data.head()
merge_data = mortality_dd.merge(depression, on='PatientID', how='right')
merge_data.head()

,PatientID,DateOfDeath,VisitID,StaffID,VisitDate,ICD10_1,ICD10_2,ICD10_3,ClinicCode,depression
0,2,2008-11-02,15,14,2006-07-04,C4650,O10019,NaN,40,False
1,4,2016-01-17,38,27,2010-03-07,J452,E131,NaN,53,False
2,13,2010-11-27,147,47,2009-01-14,Z6845,E119,C780,24,False
3,17,2008-11-27,181,2,NaN,F1014,I2781,NaN,55,False
4,18,2010-09-22,182,49,NaN,E131,NaN,NaN,50,False


In [22]:
merge_data.set_index('PatientID')
merge_data['death'] = merge_data.DateOfDeath.isnull()

In [23]:
merge_data['death'] = merge_data['death'].astype(int)
merge_data.head()

,PatientID,DateOfDeath,VisitID,StaffID,VisitDate,ICD10_1,ICD10_2,ICD10_3,ClinicCode,depression,death
0,2,2008-11-02,15,14,2006-07-04,C4650,O10019,NaN,40,False,0
1,4,2016-01-17,38,27,2010-03-07,J452,E131,NaN,53,False,0
2,13,2010-11-27,147,47,2009-01-14,Z6845,E119,C780,24,False,0
3,17,2008-11-27,181,2,NaN,F1014,I2781,NaN,55,False,0
4,18,2010-09-22,182,49,NaN,E131,NaN,NaN,50,False,0


In [24]:
merge_data.groupby(['depression']).PatientID.count().head()

/anaconda3/envs/py36/lib/python3.6/site-packages/dask/dataframe/core.py:4382: UserWarning: Insufficient elements for `head`. 5 elements requested, only 2 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


depression
False    17592
True      2056
Name: PatientID, dtype: int64

In [272]:
merge_data.groupby(['depression']).death.sum().head()/merge_data.groupby(['depression']).PatientID.count().head()

/anaconda3/envs/py36/lib/python3.6/site-packages/dask/dataframe/core.py:4382: UserWarning: Insufficient elements for `head`. 5 elements requested, only 2 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))
/anaconda3/envs/py36/lib/python3.6/site-packages/dask/dataframe/core.py:4382: UserWarning: Insufficient elements for `head`. 5 elements requested, only 2 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(msg.format(n, len(r)))


depression
False    0.656946
True     0.569553
dtype: float64

## NO. The mortality among people who have depression is about 57.0% while people who don't have depression is 65.7%.

## Q2 Repeat number 1, but this time use Apache Spark to implement your solution.